<a href="https://colab.research.google.com/github/taniahdez75/st/blob/main/Homework3_Tania.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#Feature Importance
#Dataframe for CAse MAnangement Evaluation
import pandas as pd
import numpy as np
import requests

#Visualization
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#MAchine learning model
from sklearn .linear_model import LogisticRegression




In [8]:
#Creating Member Data

import numpy as np
import pandas as pd
import datetime

# Define the number of records
num_records = 1000

# Define member_id as a sequence of integers
member_id = list(range(1, num_records + 1))

# Define sex, age, diabetes and dementia as random variables
sex = np.random.choice(['M', 'F'], size=num_records)
age = np.random.normal(50, 10, size=num_records)
diabetes = np.random.choice([0, 1], size=num_records)
dementia = np.random.choice([0, 1], size=num_records)

#SDOH
HousingIns = np.random.uniform(low=1.0, high=10.0, size=num_records)
FoodIns = np.random.uniform(low=1.0, high=20.0, size=num_records)
SVIFactor = np.random.uniform(low=1.0, high=30.0, size=num_records)
TranspIns = np.random.uniform(low=1.0, high=40.0, size=num_records)

# Define case_management_date as a random date
start_date = '2019-01-01'
end_date = '2022-12-31'
case_management_date = pd.to_datetime(np.random.choice(pd.date_range(start=start_date, end=end_date), size=num_records))

# Define the CM flag column
CM = np.random.choice([0, 1], size=num_records)

# Define risk_level as a random variable
risk_level = np.random.choice(['High', 'Medium', 'Low', 'None'], size=num_records)

# Create a dataframe with the generated columns
data = {
    'member_id': member_id,
    'sex': sex,
    'age': age,
    'diabetes': diabetes,
    'dementia': dementia,
    'HousingIns':HousingIns,
    'FoodIns':FoodIns,
    'SVIFactor':SVIFactor,
    'TranspIns':TranspIns,
    'case_management_date': case_management_date,
    'CM': CM,
    'risk_level':risk_level
}

df = pd.DataFrame(data)

# Preview the dataframe
print(df.head())

   member_id sex        age  diabetes  dementia  HousingIns    FoodIns  \
0          1   F  48.702670         0         1    3.068120  17.563781   
1          2   M  54.962374         0         0    5.297407  16.107555   
2          3   F  54.696571         1         1    4.989313   9.638523   
3          4   F  55.169737         0         0    4.224295  11.569407   
4          5   M  38.601591         0         1    2.217510   2.704904   

   SVIFactor  TranspIns case_management_date  CM risk_level  
0  17.887988  29.563134           2022-12-19   0     Medium  
1  17.195904   7.117386           2021-04-19   0     Medium  
2  13.852421  27.299715           2022-01-28   0       High  
3   9.566202  23.612140           2022-03-07   0       High  
4   1.919772  17.041419           2021-06-30   0     Medium  


In [9]:
# Creating Cost Data
#Define the number of records
num_records1 = 10000

# Get a random selection of values in df1 column A
values_A = np.random.choice(df['member_id'], size=num_records)

# Define cost_before_CM and cost_after_CM as random amounts of money
cost = np.random.normal(500, 100, size=num_records1)

# Define claim_type as a random variable
claim_type = np.random.choice(['ER', 'IP', 'RX'], size=num_records1)
period = np.random.choice(['PRE', 'POS'], size=num_records1)
  
# Create a dataframe with the generated columns
data = {
    'member_id':np.random.choice(values_A, size=num_records1),
    'period':period,
    'cost': cost,
    'claim_type': claim_type
}

df1 = pd.DataFrame(data)

# Preview the dataframe
print(df1.head())

   member_id period        cost claim_type
0        849    POS  650.316460         ER
1        275    PRE  506.226657         ER
2        757    PRE  358.435484         IP
3        319    PRE  514.125302         ER
4        196    PRE  464.832920         IP


In [12]:
# group by id, p, and ct and sum c
df2 = df1.groupby(['member_id', 'period', 'claim_type'], as_index=False)['cost'].sum()


In [13]:
# perform left join
df_left_join = pd.merge(df, df2, on='member_id', how='left')

# print result
print(df_left_join)

      member_id sex        age  diabetes  dementia  HousingIns    FoodIns  \
0             1   F  48.702670         0         1    3.068120  17.563781   
1             1   F  48.702670         0         1    3.068120  17.563781   
2             1   F  48.702670         0         1    3.068120  17.563781   
3             1   F  48.702670         0         1    3.068120  17.563781   
4             1   F  48.702670         0         1    3.068120  17.563781   
...         ...  ..        ...       ...       ...         ...        ...   
3675       1000   M  34.369628         0         1    2.383269  11.929602   
3676       1000   M  34.369628         0         1    2.383269  11.929602   
3677       1000   M  34.369628         0         1    2.383269  11.929602   
3678       1000   M  34.369628         0         1    2.383269  11.929602   
3679       1000   M  34.369628         0         1    2.383269  11.929602   

      SVIFactor  TranspIns case_management_date  CM risk_level period  \
0 